# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../.."))

In [2]:
%pip install optuna
%pip install belashovplot
%pip install timm

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


# Импорты и настройка модулей

In [3]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from utilities.training import train, confusion
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
from pickle import dump
import torch
import numpy
import timm
import optuna
import pandas

C:\Users\uclap\Documents\Coding\Python\OpticalEncoder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FigureWidthHeight = (16, 16)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA GeForce RTX 3060 Ti


# Инициализация вспомогательных методов

## Комбинированная лосс функция

In [6]:
def combined_loss(cross_entropy_to_mse_proportion:float=1.0):
    def loss_function(outputs, targets):
        CELoss = torch.nn.functional.cross_entropy(outputs, targets)
        MSELoss = torch.nn.functional.mse_loss(torch.nn.functional.softmax(outputs, dim=1), torch.nn.functional.one_hot(targets, num_classes=10).float())
        loss = cross_entropy_to_mse_proportion*CELoss + (1.0-cross_entropy_to_mse_proportion)*MSELoss
        return loss
    return loss_function

## Электронная моель ResNet

In [7]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=True,  in_chans=1, num_classes=classes).to(device)

## Инициализация системы по полному набору параметров

In [8]:
def initialize(
    wavelength:float, N:int, length:float, pixels:int, masks_amount:int, distance:float,
    spatial_coherence:float, time_coherence:float, time_relaxation:float, mean_samples:int,
    detector_size:float, detectors_amount:int,
    loss_function_proportion:float,  batch_size:int, optimizer_type:str
):
    print("Инициализированна модель со следующими параметрами:")
    print(f"\tДлинна волны:                             {engineering(wavelength, 'м')}")
    print(f"\tКоличество вычислительных пикселей:       {N}")
    print(f"\tКоличество пикселей маски:                {pixels}")
    print(f"\tРазмер оптических элементов:              {engineering(length, 'м')}")
    print(f"\tРазмер пикселя маски:                     {engineering(length/pixels, 'м')}")
    print(f"\tРасстояние между слоями:                  {engineering(distance, 'м')}")
    
    print(f"\tВременная когерентность:                  {engineering(time_coherence, 'с')}")
    print(f"\tВремя релаксации:                         {engineering(time_relaxation, 'c')}")
    print(f"\tПространственная когерентность:           {engineering(spatial_coherence, 'м')}")
    print(f"\tКоличество усреднений:                    {mean_samples}")

    print(f"\tРазмер детекторов:                        {engineering(detector_size, 'м')}")
    print(f"\tКоличество детекторов:                    {detectors_amount} на {detectors_amount}")

    print(f"\tПропорция CE к MSE лосс:                  {loss_function_proportion}")
    print(f"\tРазмер батча:                             {batch_size}")
    print(f"\tТип оптимизатора:                         {optimizer_type}")
    print()
    
    incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, N, length).to(device)

    spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
    detectors_filter = Gaussian((detector_size, detector_size), (0,0))
    detectors = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter).to(device)

    electronic = resnet18()

    propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
    phase_modulators = [PhaseModulator(N, length, pixels) for i in range(masks_amount)]
    amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
    elements = [phase_modulators[0], amplitude_modulators[0]]
    for phase_modulator, amplitude_modulator in zip(phase_modulators[1:], amplitude_modulators[1:]):
        elements.append(propagation)
        elements.append(phase_modulator)
        elements.append(amplitude_modulator)
    elements.append(propagation)
    chunker = CudaMemoryChunker(sub_chunks=mean_samples)
    optical = CompositeModel(*elements)
    optical.wrap(chunker)
    optical.wrap(incoherent)
    optical.to(device)
    model = HybridModel(optical, detectors, electronic).to(device)

    dataset = Dataset('CIFAR10', batch_size, N, N, torch.complex64)
    dataset.train
    dataset.test
    
    loss_function = combined_loss(loss_function_proportion)
    optimizer_types_list = {'Adam':torch.optim.Adam, 'SGD':torch.optim.SGD, 'RMSprop':torch.optim.RMSprop, 'Adagrad':torch.optim.Adagrad}
    optimizer_type = optimizer_types_list[optimizer_type]

    return model, dataset, loss_function, optimizer_type, (incoherent, chunker, optical, detectors, electronic)

# Установка не изменяемых параметров системы

In [9]:
# Предпочтительные параметры
size = 50.0E-6
near_N = 300
near_length = 5.0E-3
wavelength = 500.0E-9
detectors_amount = 24

# Параметры когерентности
spatial_coherence = 50.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 7

In [10]:
# Вычисляемые параметры
pixels = upper_integer(near_length/size)
length = pixels * size
cppp = upper_integer(near_N * size / length)
N = int(length / size) * cppp
detector_size = length / 60

# Отображение базы данных тренировок

In [11]:
study = optuna.create_study(study_name="ID2NN", storage="sqlite:///ID2NN.db", direction='maximize', load_if_exists=True)
dataframe:pandas.DataFrame = study.trials_dataframe()
print(f"Кол-во исследований: {len(study.trials)}")

[I 2024-05-05 03:31:37,521] Using an existing study with name 'ID2NN' instead of creating a new one.


Кол-во исследований: 275


In [28]:
dataframe.nlargest(10, 'value')

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
108,108,30.02,2024-05-02 22:18:41.949319,2024-05-02 22:30:25.700272,0 days 00:11:43.750953,36,0.931286,0.080028,0.005589,6,RMSprop,COMPLETE
268,268,30.00,2024-05-04 21:01:40.271930,2024-05-04 21:13:24.276497,0 days 00:11:44.004567,38,0.829635,0.102949,0.004682,5,Adam,COMPLETE
273,273,29.81,2024-05-04 22:01:08.745629,2024-05-04 22:13:11.440205,0 days 00:12:02.694576,38,0.797449,0.111228,0.004454,5,Adam,COMPLETE
267,267,29.49,2024-05-04 20:49:53.941394,2024-05-04 21:01:40.242940,0 days 00:11:46.301546,38,0.798459,0.103915,0.004568,5,Adam,COMPLETE
272,272,29.44,2024-05-04 21:49:27.500663,2024-05-04 22:01:08.706298,0 days 00:11:41.205635,38,0.789457,0.112727,0.004565,5,Adam,COMPLETE
109,109,29.42,2024-05-02 22:30:25.719252,2024-05-02 22:42:09.999907,0 days 00:11:44.280655,36,0.992874,0.080766,0.005603,6,RMSprop,COMPLETE
262,262,29.29,2024-05-04 19:52:31.046571,2024-05-04 20:04:16.509982,0 days 00:11:45.463411,37,0.785131,0.103529,0.003100,5,Adam,COMPLETE
265,265,29.04,2024-05-04 20:26:50.107605,2024-05-04 20:38:10.500097,0 days 00:11:20.392492,38,0.793974,0.103155,0.004389,5,Adam,COMPLETE
243,243,29.03,2024-05-04 16:33:44.271063,2024-05-04 16:43:33.993515,0 days 00:09:49.722452,36,0.913746,0.096982,0.002657,5,Adam,COMPLETE
261,261,28.86,2024-05-04 19:41:01.919909,2024-05-04 19:52:31.020580,0 days 00:11:29.100671,37,0.839044,0.104015,0.003080,5,Adam,COMPLETE


In [26]:
dataframe[dataframe['value'] >= 29].nlargest(10, 'params_distance')

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
272,272,29.44,2024-05-04 21:49:27.500663,2024-05-04 22:01:08.706298,0 days 00:11:41.205635,38,0.789457,0.112727,0.004565,5,Adam,COMPLETE
273,273,29.81,2024-05-04 22:01:08.745629,2024-05-04 22:13:11.440205,0 days 00:12:02.694576,38,0.797449,0.111228,0.004454,5,Adam,COMPLETE
267,267,29.49,2024-05-04 20:49:53.941394,2024-05-04 21:01:40.242940,0 days 00:11:46.301546,38,0.798459,0.103915,0.004568,5,Adam,COMPLETE
262,262,29.29,2024-05-04 19:52:31.046571,2024-05-04 20:04:16.509982,0 days 00:11:45.463411,37,0.785131,0.103529,0.003100,5,Adam,COMPLETE
265,265,29.04,2024-05-04 20:26:50.107605,2024-05-04 20:38:10.500097,0 days 00:11:20.392492,38,0.793974,0.103155,0.004389,5,Adam,COMPLETE
268,268,30.00,2024-05-04 21:01:40.271930,2024-05-04 21:13:24.276497,0 days 00:11:44.004567,38,0.829635,0.102949,0.004682,5,Adam,COMPLETE
243,243,29.03,2024-05-04 16:33:44.271063,2024-05-04 16:43:33.993515,0 days 00:09:49.722452,36,0.913746,0.096982,0.002657,5,Adam,COMPLETE
109,109,29.42,2024-05-02 22:30:25.719252,2024-05-02 22:42:09.999907,0 days 00:11:44.280655,36,0.992874,0.080766,0.005603,6,RMSprop,COMPLETE
108,108,30.02,2024-05-02 22:18:41.949319,2024-05-02 22:30:25.700272,0 days 00:11:43.750953,36,0.931286,0.080028,0.005589,6,RMSprop,COMPLETE


# Модель №1

In [27]:
dataframe[dataframe['number'] == 268]

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
268,268,30.0,2024-05-04 21:01:40.271930,2024-05-04 21:13:24.276497,0 days 00:11:44.004567,38,0.829635,0.102949,0.004682,5,Adam,COMPLETE


## Утсановка оптимизированных параметров системы

In [29]:
masks_amount = 5
batch_size = 38
distance = 0.1
learning_rate = 0.0047
loss_function_proportion = 0.83
optimizer_type = 'Adam'

In [30]:
model, dataset, loss_function, optimizer_type, (incoherent, chunker, optical, detectors, electronic) = initialize(
    wavelength, N, length, pixels, masks_amount, distance,
    spatial_coherence, time_coherence, time_relaxation, mean_samples,
    detector_size, detectors_amount,
    loss_function_proportion,  batch_size, optimizer_type
)

Инициализированна модель со следующими параметрами:
	Длинна волны:                             500.0 нм
	Количество вычислительных пикселей:       300
	Количество пикселей маски:                100
	Размер оптических элементов:              5.0 мм
	Размер пикселя маски:                     50.0 мкм
	Расстояние между слоями:                  100.0 мм
	Временная когерентность:                  10.0 нс
	Время релаксации:                         1.0 мкc
	Пространственная когерентность:           50.0 мкм
	Количество усреднений:                    7
	Размер детекторов:                        83.333 мкм
	Количество детекторов:                    24 на 24
	Пропорция CE к MSE лосс:                  0.83
	Размер батча:                             38
	Тип оптимизатора:                         Adam

Files already downloaded and verified
Files already downloaded and verified


## Тренировка

In [31]:
loss_histories = []
cofusion_matrixes = []

In [32]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate)
cofusion_matrixes.append(confusion(model, dataset))
print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")
for i in range(30):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:55<00:00,  4.77it/s]


Точность: 10.0%


RLoss: 1.8761957765896906, RPI1000: -0.0333958314242043: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [08:57<00:00,  2.45it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.89it/s]


Точность: 15.1%


RLoss: 1.793234761947816, RPI1000: -0.0478075157397006: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:06<00:00,  2.41it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.93it/s]


...

Точность: 40.03%


RLoss: 1.490924111527167, RPI1000: 0.011162067307486334: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:05<00:00,  2.41it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.97it/s]


Точность: 39.03%


RLoss: 1.3552342440118064, RPI1000: 0.0019030364479546594: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:05<00:00,  2.41it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.94it/s]

Точность: 38.4%


In [33]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate/100)
for i in range(30):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

RLoss: 1.301670629734995, RPI1000: -0.024914056902260978: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:07<00:00,  2.41it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.94it/s]


Точность: 40.3%


RLoss: 1.438901318135695, RPI1000: -0.005007009728102194: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:06<00:00,  2.41it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.93it/s]


Точность: 40.66%


RLoss: 1.37697981577835, RPI1000: -0.0007854494406694627: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:04<00:00,  2.42it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.97it/s]


...

Точность: 41.39%


RLoss: 1.339557425368997, RPI1000: 0.00594153956887741: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:03<00:00,  2.42it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.98it/s]


Точность: 41.55%


RLoss: 1.2715889059391832, RPI1000: 0.004648382729553037: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1316/1316 [09:04<00:00,  2.42it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:53<00:00,  4.96it/s]

Точность: 41.83%
